In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
data = pd.read_csv("../data_cancer_tratado.csv",sep=';', encoding='utf-8')

In [3]:
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883


In [4]:
data.tail()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016
568,92751,B,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,0.1587,0.05884


In [5]:
data.dtypes

id                          int64
diagnosis                  object
radius_mean               float64
texture_mean              float64
perimeter_mean            float64
area_mean                 float64
smoothness_mean           float64
compactness_mean          float64
concavity_mean            float64
concave points_mean       float64
symmetry_mean             float64
fractal_dimension_mean    float64
dtype: object

In [6]:
data.shape

(569, 12)

# Transformando variavéis catégoricas nominais em ordinais 

In [7]:
data2 = pd.DataFrame.copy(data)

In [8]:
data2["diagnosis"].replace({"M": 0, "B": 1},inplace = True)

In [9]:
data2.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
0,842302,0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871
1,842517,0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667
2,84300903,0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999
3,84348301,0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744
4,84358402,0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883


In [10]:
data2.dtypes

id                          int64
diagnosis                   int64
radius_mean               float64
texture_mean              float64
perimeter_mean            float64
area_mean                 float64
smoothness_mean           float64
compactness_mean          float64
concavity_mean            float64
concave points_mean       float64
symmetry_mean             float64
fractal_dimension_mean    float64
dtype: object

In [11]:
previsores = data2.iloc[:,2:12].values
previsores

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 1.471e-01, 2.419e-01,
        7.871e-02],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 7.017e-02, 1.812e-01,
        5.667e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 1.279e-01, 2.069e-01,
        5.999e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 5.302e-02, 1.590e-01,
        5.648e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 1.520e-01, 2.397e-01,
        7.016e-02],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 1.587e-01,
        5.884e-02]])

In [12]:
previsores.shape

(569, 10)

In [13]:
alvo = data2.iloc[:,1].values

In [14]:
alvo.shape

(569,)

In [15]:
data2.describe()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,0.627417,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798
std,1.250206e+08,0.483918,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060
min,8.670000e+03,0.000000,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960
25%,8.692180e+05,0.000000,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700
50%,9.060240e+05,1.000000,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540
75%,8.813129e+06,1.000000,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120
max,9.113205e+08,1.000000,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440


In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [18]:
previsores_esc

array([[ 1.09706398e+00, -2.07333501e+00,  1.26993369e+00, ...,
         2.53247522e+00,  2.21751501e+00,  2.25574689e+00],
       [ 1.82982061e+00, -3.53632408e-01,  1.68595471e+00, ...,
         5.48144156e-01,  1.39236330e-03, -8.68652457e-01],
       [ 1.57988811e+00,  4.56186952e-01,  1.56650313e+00, ...,
         2.03723076e+00,  9.39684817e-01, -3.98007910e-01],
       ...,
       [ 7.02284249e-01,  2.04557380e+00,  6.72675785e-01, ...,
         1.05777359e-01, -8.09117071e-01, -8.95586935e-01],
       [ 1.83834103e+00,  2.33645719e+00,  1.98252415e+00, ...,
         2.65886573e+00,  2.13719425e+00,  1.04369542e+00],
       [-1.80840125e+00,  1.22179204e+00, -1.81438851e+00, ...,
        -1.26181958e+00, -8.20069901e-01, -5.61032377e-01]])

# **BASE DE TREINO E TESTE**

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
x_treino, x_teste, y_treino, y_teste = train_test_split(previsores_esc, alvo, test_size = 0.3, random_state = 7)

In [21]:
x_treino.shape

(398, 10)

In [22]:
x_teste.shape

(171, 10)

In [23]:
y_treino.shape

(398,)

In [24]:
y_teste.shape

(171,)

# Naive Bayes

In [25]:
from sklearn.naive_bayes import GaussianNB

In [26]:
naive = GaussianNB()
naive.fit(x_treino, y_treino)

GaussianNB()

In [27]:
previsoes_naive = naive.predict(x_teste)
previsoes_naive

array([1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0], dtype=int64)

In [28]:
y_teste

array([1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0], dtype=int64)

In [29]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [30]:
accuracy_score(y_teste, previsoes_naive)

0.9298245614035088

In [31]:
print("Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_naive) * 100.0))

Acurácia: 92.98%


In [32]:
confusion_matrix(y_teste, previsoes_naive)

array([[ 47,   8],
       [  4, 112]], dtype=int64)

**Análise dados treino**

In [33]:
previsoes_treino = naive.predict(x_treino)
previsoes_treino

array([1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,

In [34]:
accuracy_score(y_treino, previsoes_treino)

0.907035175879397

In [35]:
print("Acurácia: %.2f%%" % (accuracy_score(y_treino, previsoes_treino) * 100.0))

Acurácia: 90.70%


In [36]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [37]:
kfold = KFold(n_splits = 30, shuffle=True, random_state = 7)

In [38]:
modelo = GaussianNB()
resultado = cross_val_score(modelo, previsores, alvo, cv = kfold)
resultado

array([0.94736842, 0.84210526, 0.94736842, 1.        , 0.89473684,
       0.94736842, 0.89473684, 1.        , 0.84210526, 1.        ,
       0.89473684, 1.        , 0.89473684, 0.94736842, 0.89473684,
       0.84210526, 0.89473684, 0.78947368, 0.94736842, 0.89473684,
       0.94736842, 0.94736842, 0.78947368, 0.94736842, 0.84210526,
       0.84210526, 0.94736842, 1.        , 0.78947368, 0.94444444])

In [39]:
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

Acurácia Média: 91.04%


Naive Bayes = 92,98% (treino e teste) - 159 acertos e 90,95% (validação cruzada) - previsores

# **MÁQUINAS DE VETORES DE SUPORTE (SVM)**

In [40]:
from sklearn.svm import SVC

In [41]:
param_grid = dict(
    C=[1, 2, 3, 4, 5],
    kernel=['linear', 'poly', 'rbf', 'sigmoid']
    )

In [42]:
svm = SVC(random_state=1)

In [43]:
grid_search = GridSearchCV(svm, param_grid, scoring='roc_auc', cv=4)

In [44]:
grid_search.fit(x_treino, y_treino)

GridSearchCV(cv=4, estimator=SVC(random_state=1),
             param_grid={'C': [1, 2, 3, 4, 5],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='roc_auc')

In [45]:
grid_search.best_params_

{'C': 3, 'kernel': 'rbf'}

In [46]:
resultado = pd.DataFrame(grid_search.cv_results_)
print(resultado.shape)
resultado.head(3)

(20, 14)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002494,4.988318e-04,0.002493,0.000499,1,linear,"{'C': 1, 'kernel': 'linear'}",0.983186,0.988750,0.976923,0.982051,0.982728,0.004202,6
1,0.001995,8.492355e-07,0.002244,0.000432,1,poly,"{'C': 1, 'kernel': 'poly'}",0.976881,0.982500,0.971368,0.974786,0.976384,0.004043,15
2,0.002494,8.637947e-04,0.002743,0.000432,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.987390,0.989167,0.966667,0.994017,0.984310,0.010471,5


In [47]:
resultado.sort_values(by='mean_test_score', ascending=False, inplace=True)

resultado.reset_index(drop=True, inplace=True)

resultado[['param_C', 'param_kernel',
    'mean_test_score', 'std_test_score']].head()

,param_C,param_kernel,mean_test_score,std_test_score
0,3,rbf,0.986773,0.006200
1,2,rbf,0.985913,0.008085
2,4,rbf,0.985815,0.007078
3,5,rbf,0.984744,0.008963
4,1,rbf,0.984310,0.010471


In [48]:
svm = SVC(kernel='rbf', random_state=7, C = 2)
svm.fit(x_treino, y_treino)

SVC(C=2, random_state=7)

In [49]:
svm.fit(x_treino, y_treino)

SVC(C=2, random_state=7)

In [50]:
previsoes_svm = svm.predict(x_teste)
previsoes_svm

array([1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0], dtype=int64)

In [51]:
y_teste

array([1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0], dtype=int64)

In [52]:
print("Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_svm) * 100.0))

Acurácia: 95.32%


In [53]:
confusion_matrix(y_teste, previsoes_svm)

array([[ 47,   8],
       [  0, 116]], dtype=int64)

In [54]:
print(classification_report(y_teste, previsoes_svm))

              precision    recall  f1-score   support

           0       1.00      0.85      0.92        55
           1       0.94      1.00      0.97       116

    accuracy                           0.95       171
   macro avg       0.97      0.93      0.94       171
weighted avg       0.96      0.95      0.95       171



**Análise dados treino**

In [55]:
previsoes_treino_svm = svm.predict(x_treino)
previsoes_treino_svm

array([1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,

In [56]:
y_treino

array([1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,

In [57]:
accuracy_score(y_treino, previsoes_treino_svm)

0.957286432160804

In [58]:
kfold = KFold(n_splits = 30, shuffle=True, random_state = 7)

In [59]:
modelo = SVC(kernel='rbf', random_state=7, C = 2)
resultado = cross_val_score(modelo, previsores_esc, alvo, cv = kfold)

# Usamos a média e o desvio padrão
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

Acurácia Média: 95.60%


Naive Bayes = 92,98% (treino e teste) - 159 acertos e 90,95% (validação cruzada) - previsores

SVM = 95.32% (treino e teste) - 163 acertos e 95,60% (validação cruzada) - previsores_esc - SVC(kernel='rbf', random_state=7, C = 2)

# **REGRESSÃO LOGÍSTICA**

In [60]:
from sklearn.linear_model import LogisticRegression

In [61]:
param_grid = dict(
    max_iter=[100, 200, 300, 400, 500],
    solver=['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    C=[1,2,3,4,5],
    penalty=['l1', 'l2', 'elasticnet', None],
    )

In [62]:
logistica = LogisticRegression(random_state=7)

In [63]:
grid_search = GridSearchCV(logistica, param_grid, scoring='roc_auc', cv=4)

In [64]:
grid_search.fit(x_treino, y_treino)

GridSearchCV(cv=4, estimator=LogisticRegression(random_state=7),
             param_grid={'C': [1, 2, 3, 4, 5],
                         'max_iter': [100, 200, 300, 400, 500],
                         'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='roc_auc')

In [65]:
grid_search.best_params_

{'C': 2, 'max_iter': 100, 'penalty': 'l2', 'solver': 'saga'}

In [66]:
resultado = pd.DataFrame(grid_search.cv_results_)
print(resultado.shape)
resultado.head(3)

(600, 16)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_max_iter,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000499,4.990101e-04,0.000000,0.000000,1,100,l1,lbfgs,"{'C': 1, 'max_iter': 100, 'penalty': 'l1', 'so...",NaN,NaN,NaN,NaN,NaN,NaN,326
1,0.001995,2.920019e-07,0.002244,0.001087,1,100,l1,liblinear,"{'C': 1, 'max_iter': 100, 'penalty': 'l1', 'so...",0.982766,0.98875,0.976923,0.978205,0.981661,0.004633,70
2,0.000499,4.988315e-04,0.000000,0.000000,1,100,l1,newton-cg,"{'C': 1, 'max_iter': 100, 'penalty': 'l1', 'so...",NaN,NaN,NaN,NaN,NaN,NaN,326


In [67]:
resultado.sort_values(by='mean_test_score', ascending=False, inplace=True)

resultado.reset_index(drop=True, inplace=True)

resultado[['param_C', 'param_max_iter', 'param_penalty', 'param_solver',
    'mean_test_score', 'std_test_score']].head()

,param_C,param_max_iter,param_penalty,param_solver,mean_test_score,std_test_score
0,2,100,l2,saga,0.983045,0.004838
1,1,500,l2,newton-cholesky,0.983045,0.005163
2,1,300,l2,newton-cg,0.983045,0.005163
3,1,500,l2,sag,0.983045,0.005163
4,1,500,l2,newton-cg,0.983045,0.005163


In [68]:
logistica = LogisticRegression(random_state=7, max_iter=100, penalty=None,
                               tol=0.0001, C=1,solver="newton-cg")
logistica.fit(x_treino, y_treino)

LogisticRegression(C=1, penalty=None, random_state=7, solver='newton-cg')

In [69]:
previsoes_logistica = logistica.predict(x_teste)
previsoes_logistica

array([1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0], dtype=int64)

In [70]:
print("Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_logistica) * 100.0))

Acurácia: 94.74%


In [71]:
confusion_matrix(y_teste, previsoes_logistica)

array([[ 50,   5],
       [  4, 112]], dtype=int64)

In [72]:
print(classification_report(y_teste, previsoes_logistica))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92        55
           1       0.96      0.97      0.96       116

    accuracy                           0.95       171
   macro avg       0.94      0.94      0.94       171
weighted avg       0.95      0.95      0.95       171



**Análise dados treino**

In [73]:
previsoes_treino_logistica = logistica.predict(x_treino)
previsoes_treino_logistica

array([1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,

In [74]:
print("Acurácia: %.2f%%" % (accuracy_score(y_treino, previsoes_treino_logistica) * 100.0))

Acurácia: 94.72%


In [75]:
kfold = KFold(n_splits = 30, shuffle=True, random_state = 7)

In [76]:
modelo = LogisticRegression(random_state=7, max_iter=200, penalty="l2",
                               tol=0.0001, C=1,solver="lbfgs")
resultado = cross_val_score(modelo, previsores, alvo, cv = kfold)



In [77]:
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

Acurácia Média: 91.22%


Naive Bayes = 92,98% (treino e teste) - 159 acertos e 90,95% (validação cruzada) - previsores

SVM = 95.32% (treino e teste) - 163 acertos e 95,60% (validação cruzada) - previsores_esc - SVC(kernel='rbf', random_state=7, C = 2)

Regressao logistica = 94,74% (treino e teste) - 162 acertos e 91,92% (validação cruzada)-  previsores_esc - LogisticRegression(random_state=7, max_iter=100, penalty=None,
                               tol=0.0001, C=1,solver="newton-cg")

# ÁRVORE DE DECISÃO

In [78]:
from sklearn.tree import DecisionTreeClassifier

In [79]:
param_grid = dict(
    max_depth=[1,2,3,4,5],
    criterion=['gini', 'entropy']
    )

In [80]:
arvore = DecisionTreeClassifier(random_state = 7)

In [81]:
grid_search = GridSearchCV(arvore, param_grid, scoring='roc_auc', cv=4)

In [82]:
grid_search.fit(x_treino, y_treino)

GridSearchCV(cv=4, estimator=DecisionTreeClassifier(random_state=7),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 2, 3, 4, 5]},
             scoring='roc_auc')

In [83]:
grid_search.best_params_

{'criterion': 'entropy', 'max_depth': 3}

In [84]:
resultado = pd.DataFrame(grid_search.cv_results_)
print(resultado.shape)
resultado.head(3)

(10, 14)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001994,7.057914e-04,0.001746,0.000432,gini,1,"{'criterion': 'gini', 'max_depth': 1}",0.907734,0.875000,0.857051,0.919872,0.889914,0.025087,9
1,0.002244,4.316738e-04,0.001497,0.000499,gini,2,"{'criterion': 'gini', 'max_depth': 2}",0.921185,0.939167,0.870299,0.934829,0.916370,0.027414,5
2,0.002992,2.665601e-07,0.001746,0.000431,gini,3,"{'criterion': 'gini', 'max_depth': 3}",0.866330,0.938750,0.814316,0.942521,0.890479,0.053438,8


In [85]:
resultado.sort_values(by='mean_test_score', ascending=False, inplace=True)

resultado.reset_index(drop=True, inplace=True)

resultado[['param_criterion', 'param_max_depth',
    'mean_test_score', 'std_test_score']].head()

,param_criterion,param_max_depth,mean_test_score,std_test_score
0,entropy,3,0.937781,0.040205
1,entropy,4,0.935421,0.030808
2,entropy,5,0.929726,0.026680
3,entropy,2,0.927513,0.034348
4,gini,2,0.916370,0.027414


In [86]:
arvore = DecisionTreeClassifier(random_state = 7, criterion="entropy",max_depth=3)

In [87]:
arvore.fit(x_treino,y_treino)

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=7)

In [88]:
previsao_arvore = arvore.predict(x_teste)

In [89]:
previsao_arvore

array([1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0], dtype=int64)

In [90]:
y_teste

array([1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0], dtype=int64)

In [91]:
print("Acurácia: %.2f%%" % (accuracy_score(y_teste, previsao_arvore) * 100.0))

Acurácia: 91.81%


In [92]:
confusion_matrix(y_teste, previsao_arvore)

array([[ 42,  13],
       [  1, 115]], dtype=int64)

In [93]:
print(classification_report(y_teste, previsao_arvore))

              precision    recall  f1-score   support

           0       0.98      0.76      0.86        55
           1       0.90      0.99      0.94       116

    accuracy                           0.92       171
   macro avg       0.94      0.88      0.90       171
weighted avg       0.92      0.92      0.92       171



**Análise dados treino**

In [94]:
previsao_arvore_treino= arvore.predict(x_treino)
previsao_arvore_treino

array([1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,

In [95]:
y_treino

array([1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,

In [96]:
print("Acurácia: %.2f%%" % (accuracy_score(y_treino, previsao_arvore_treino) * 100.0))

Acurácia: 93.72%


In [97]:
kfold = KFold(n_splits = 30, shuffle=True, random_state = 7)

In [98]:
modelo = DecisionTreeClassifier(random_state = 7, criterion="entropy",max_depth=3)
resultado = cross_val_score(modelo, previsores, alvo, cv = kfold)


In [99]:
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

Acurácia Média: 89.99%


Naive Bayes = 92,98% (treino e teste) - 159 acertos e 90,95% (validação cruzada) - previsores

**SVM = 95.32% (treino e teste) - 163 acertos e 95,60% (validação cruzada) - previsores_esc - SVC(kernel='rbf', random_state=7, C = 2)**

Regressao logistica = 94,74% (treino e teste) - 162 acertos e 91,92% (validação cruzada)- previsores_esc - LogisticRegression(random_state=7, max_iter=100, penalty=None, tol=0.0001, C=1,solver="newton-cg")

Árvore de decisão - 91.81% treino 93.72% teste - 157 acertos e 89,99% (validação cruzada)- previsores previsores_esc - DecisionTreeClassifier(random_state = 7, criterion="entropy",max_depth=3) 

In [100]:
param_grid = dict(
    hidden_layer_sizes=[6, 7, 8],
    activation=['identity', 'logistic', 'tanh', 'relu'],
    solver=['lbfgs', 'sgd', 'adam'],
    max_iter=[100, 200, 400, 600, 800,1000]
    )

In [102]:
from sklearn.neural_network import MLPClassifier

In [104]:
x_treino_rn, x_teste_rn, y_treino_rn, y_teste_rn = train_test_split(previsores, alvo, test_size = 0.3, random_state = 7)

In [105]:
rede = MLPClassifier(random_state=7)

grid_search = GridSearchCV(rede, param_grid, scoring='roc_auc', cv=4)

grid_search.fit(x_treino_rn, y_treino_rn)

grid_search.best_params_

{'activation': 'identity',
 'hidden_layer_sizes': 8,
 'max_iter': 1000,
 'solver': 'lbfgs'}

In [106]:
resultado = pd.DataFrame(grid_search.cv_results_)
print(resultado.shape)
resultado.head(3)

(216, 16)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_max_iter,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.022191,0.011871,0.001496,0.000498,identity,6,100,lbfgs,"{'activation': 'identity', 'hidden_layer_sizes...",0.492063,0.115874,0.500000,0.070619,0.294639,0.202047,199
1,0.020695,0.004254,0.001496,0.000499,identity,6,100,sgd,"{'activation': 'identity', 'hidden_layer_sizes...",0.909481,0.841256,0.878814,0.929817,0.889842,0.033414,103
2,0.009724,0.000431,0.001745,0.000432,identity,6,100,adam,"{'activation': 'identity', 'hidden_layer_sizes...",0.944230,0.918930,0.926765,0.970357,0.940071,0.019739,21


In [107]:
resultado.sort_values(by='mean_test_score', ascending=False, inplace=True)

resultado.reset_index(drop=True, inplace=True)

resultado[['param_activation','param_hidden_layer_sizes','param_max_iter','param_solver',
    'mean_test_score', 'std_test_score']].head()

,param_activation,param_hidden_layer_sizes,param_max_iter,param_solver,mean_test_score,std_test_score
0,identity,8,1000,lbfgs,0.981042,0.014465
1,identity,8,800,lbfgs,0.980938,0.015067
2,identity,7,800,lbfgs,0.980718,0.014783
3,identity,7,600,lbfgs,0.980614,0.014629
4,tanh,7,600,lbfgs,0.980384,0.014073


In [108]:
from sklearn.neural_network import MLPClassifier

redes = MLPClassifier(hidden_layer_sizes=(8), activation='identity', solver='lbfgs', max_iter =1000,
                              tol=0.0001, random_state = 7, verbose = True)

In [109]:
redes.fit(x_treino_rn, y_treino_rn)

MLPClassifier(activation='identity', hidden_layer_sizes=8, max_iter=1000,
              random_state=7, solver='lbfgs', verbose=True)

In [110]:
previsoes = redes.predict(x_teste_rn)

In [111]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Acurácia: %.2f%%" % (accuracy_score(y_teste_rn, previsoes) * 100.0))

Acurácia: 93.57%


In [112]:
confusion_matrix(y_teste_rn, previsoes)

array([[ 58,   5],
       [  6, 102]], dtype=int64)

In [113]:
print(classification_report(y_teste_rn, previsoes))

              precision    recall  f1-score   support

           0       0.91      0.92      0.91        63
           1       0.95      0.94      0.95       108

    accuracy                           0.94       171
   macro avg       0.93      0.93      0.93       171
weighted avg       0.94      0.94      0.94       171



In [114]:
previsoes_treino = redes.predict(x_treino_rn)

In [115]:
accuracy_score(y_treino_rn, previsoes_treino)

0.9271356783919598

In [116]:
confusion_matrix(y_treino_rn, previsoes_treino)

array([[130,  19],
       [ 10, 239]], dtype=int64)

In [117]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kfold = KFold(n_splits = 30, shuffle=True, random_state = 5)

modelo = MLPClassifier(hidden_layer_sizes=(8), activation='identity', solver='lbfgs', max_iter =1000,
                              tol=0.0001, random_state = 7, verbose = True)

In [118]:
resultado = cross_val_score(modelo, previsores, alvo, cv = kfold)
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

Acurácia Média: 92.24%


Naive Bayes = 92,98% (treino e teste) - 159 acertos e 90,95% (validação cruzada) - previsores

**SVM = 95.32% (treino e teste) - 163 acertos e 95,60% (validação cruzada) - previsores_esc - SVC(kernel='rbf', random_state=7, C = 2)**

Regressao logistica = 94,74% (treino e teste) - 162 acertos e 91,92% (validação cruzada)- previsores_esc - LogisticRegression(random_state=7, max_iter=100, penalty=None, tol=0.0001, C=1,solver="newton-cg")

Árvore de decisão - 91.81% treino 93.72% teste - 157 acertos e 89,99% (validação cruzada)- previsores previsores_esc - DecisionTreeClassifier(random_state = 7, criterion="entropy",max_depth=3) 

Redes Neurais Artificiais = 93,57% (treino e teste) - 160 acertos e 92.24% (validação cruzada) : previsores - MLPClassifier(hidden_layer_sizes=(8), activation='identity', solver='lbfgs', max_iter =1000, tol=0.0001, random_state = 7, verbose = True)